## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 9/13/2024)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from yfinance import Ticker
from pykalman import KalmanFilter

from src import elt as et

%load_ext autoreload
%autoreload 2

In [3]:
def download(symbol,interval,period):
    stock = Ticker(symbol)
    stock_df = stock.history(interval=interval,
                             period=period,
                             auto_adjust=False,
                             prepost=True, # include aftermarket hours
                            )
    stock_df.columns = stock_df.columns.str.lower().str.replace(' ', '_')
    stock_df.to_pickle(f'./data/{symbol}_{interval}_df.pkl')
    
def load(symbol,interval):
    return pd.read_pickle(f'./data/{symbol}_{interval}_df.pkl')

def transform(df):
    ...

In [4]:
download('NVDA','30m','1mo')
df = load('NVDA','30m')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 642 entries, 2024-08-16 05:00:00-04:00 to 2024-09-16 05:00:00-04:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          641 non-null    float64
 1   high          641 non-null    float64
 2   low           641 non-null    float64
 3   close         641 non-null    float64
 4   adj_close     641 non-null    float64
 5   volume        642 non-null    int64  
 6   dividends     642 non-null    float64
 7   stock_splits  642 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 45.1 KB


In [5]:
df.columns

Index(['open', 'high', 'low', 'close', 'adj_close', 'volume', 'dividends',
       'stock_splits'],
      dtype='object')